# url-prediction-model
SanjayKAroraPhD@gmail.com
October 2018

## Description
This script runs url prediction with a simple train-test split.  It assumes as input a matrix of known firm urls and potential matches from MS Bing.  Variables include the search result # (i.e., rank), length of the candidate url, matches of words derived from the known firm name and name of the url, etc. 

In [25]:
# import data processing and other libraries
import csv
import sys
import requests
import re
import pprint
import pymongo
import traceback
from time import sleep
import requests
import pandas as pd
from IPython.display import display
import time
import numpy as np

In [41]:
# import sklearn
from sklearn.model_selection import train_test_split
from sklearn import svm
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GroupKFold

In [4]:
train_df = pd.read_csv('/Users/sarora/dev/EAGER/data/training/urls/bing-firm-url-train-v5.csv')
train_df

,firm,firm_length,url,name_clnd,name_length,hit_url,hit_url_length,rank,matches,public,acquired_merged,outcome
0,honeywell international inc.,9,https://www.honeywell.com/,Honeywell - Official Site,25,https://www.honeywell.com/,26,1,1,0,0,1
1,honeywell international inc.,9,https://www.honeywell.com/,Honeywell International Inc. Company Profile |...,54,http://www.hoovers.com/company-information/cs/...,111,3,1,0,0,0
2,honeywell international inc.,9,https://www.honeywell.com/,HON:New York Stock Quote - Honeywell Internati...,58,https://www.bloomberg.com/quote/HON:US,38,4,1,1,0,0
3,honeywell international inc.,9,https://www.honeywell.com/,HON Stock Price - Honeywell International Inc....,62,https://www.marketwatch.com/investing/stock/hon,47,5,1,2,0,0
4,honeywell international inc.,9,https://www.honeywell.com/,HON Stock Price & News - Honeywell Internation...,56,https://quotes.wsj.com/HON,26,6,1,3,0,0
5,honeywell international inc.,9,https://www.honeywell.com/,Is Honeywell International Inc. a Buy? -- The ...,57,https://www.fool.com/investing/2018/10/06/is-h...,83,7,1,4,0,0
6,imds corporation,4,http://www.imds-ohio.com/,IMDS Corporation,16,http://www.imds-ohio.com/,25,1,1,0,0,1
7,imds corporation,4,http://www.imds-ohio.com/,Integrated Marketing & Distribution Services C...,50,http://imds.com.ph/,19,3,0,0,0,0
8,imds corporation,4,http://www.imds-ohio.com/,IMDS Information Pages - New to IMDS,36,https://public.mdsystem.com/web/imds-public-pa...,58,4,1,0,0,0
9,imds corporation,4,http://www.imds-ohio.com/,IMDS Information Pages - News 2016,34,https://public.mdsystem.com/web/imds-public-pa...,59,5,1,0,0,0


In [67]:
# create training and test sets, split on group (k-cross)

# X, y
X = train_df.drop(['firm', 'url', 'name_clnd', 'hit_url', 'outcome'], axis=1)
y = train_df[['outcome']]

# Check how unbalanced we are
display("Outcomes are unbalanced.")
unique, counts = np.unique(y, return_counts=True)
display(dict(zip(unique, counts)))

# Assign gruops based on 'firm'
groups = train_df.groupby('firm').ngroup().values

# Create training and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
display("Shapes of training and test data.")
display(X_train.shape)
display(X_test.shape)
display (y_train.shape)
display (y_test.shape)

avg_score = np.mean(score_array)
print(avg_score)

'Outcomes are unbalanced.'

{0: 671, 1: 99}

'Shapes of training and test data.'

(515, 7)

(255, 7)

(515, 1)

(255, 1)

In [36]:
# train a few models

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "SVC", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    SVC(gamma=0.001, C=100.), 
    QuadraticDiscriminantAnalysis()]

In [37]:
for name, clf in zip(names, classifiers):
    display (name)
    clf.fit(X_train, np.ravel(y_train))
    y_hat = clf.predict(X_test)
    y_test_numpy = y_test['outcome'].values
    # display(type(y_hat))
    # display(type(y_test_numpy))
    # display(y_hat[0:5])
    # display(y_test_numpy[0:5])
    display (pd.crosstab(y_test_numpy, y_hat, rownames=['True'], colnames=['Predicted'], margins=True))

'Nearest Neighbors'

Predicted,0,1,All
True,,,
0,208,16,224
1,6,25,31
All,214,41,255


'Linear SVM'

Predicted,0,1,All
True,,,
0,214,10,224
1,1,30,31
All,215,40,255


'RBF SVM'

...

Predicted,0,1,All
True,,,
0,209,15,224
1,1,30,31
All,210,45,255


'SVC'

Predicted,0,1,All
True,,,
0,218,6,224
1,2,29,31
All,220,35,255


'QDA'

Predicted,0,1,All
True,,,
0,210,14,224
1,1,30,31
All,211,44,255


In [50]:
X = np.array([[5, 2], [3, 4], [1, 2], [3, 4], [5,6], [7,8]])
display(X)
rkf = RepeatedKFold(n_splits=2, n_repeats=3)
for train, test in rkf.split(X):
    print("%s %s" % (train, test))

array([[5, 2],
       [3, 4],
       [1, 2],
       [3, 4],
       [5, 6],
       [7, 8]])

[0 1 5] [2 3 4]
[2 3 4] [0 1 5]
[1 2 3] [0 4 5]
[0 4 5] [1 2 3]
[0 2 4] [1 3 5]
[1 3 5] [0 2 4]
